In [2]:
import os
import sys
sys.path.append("../")

## 경고 무시
import warnings
warnings.filterwarnings('ignore')

## 시각화 툴
import matplotlib.pyplot as plt
import seaborn as sns

## encoding 
from sklearn.preprocessing import LabelEncoder

## vocabulary
import tensorflow_datasets as tfds

## preprocessor
from custom_preprocessor import Preprocessor

In [4]:
preprocessor = Preprocessor()
preprocessed_train, preprocessed_test = preprocessor.preprocess()

In [59]:
preprocessed_train["conversation"].apply(lambda x: len(x.split())).max()

239

## Label Encoding

In [6]:
## CLASS_NAMES에 '일반 대화'를 포함시킴
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']

# 수동 매핑 설정
class_mapping = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

# 'class' 열을 수동 매핑 적용하기 전에 문자열로 변환
preprocessed_train['class'] = preprocessed_train['class'].astype(str).map(class_mapping)

## Tokenizer

In [63]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

In [69]:
# BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 특수 토큰 ID 확인
CLS_TOKEN_ID = tokenizer.cls_token_id  # [CLS] 토큰 ID
SEP_TOKEN_ID = tokenizer.sep_token_id  # [SEP] 토큰 ID
UNK_TOKEN_ID = tokenizer.unk_token_id  # [UNK] 토큰 ID
PAD_TOKEN_ID = tokenizer.pad_token_id  # [PAD] 토큰 ID

# 각 문장에 [CLS], [SEP] 추가
def add_special_tokens(tokenized_texts):
    return [[CLS_TOKEN_ID] + tokenizer.convert_tokens_to_ids(tokens) + [SEP_TOKEN_ID] for tokens in tokenized_texts]

## 토크나이징
tokenized_train = [tokenizer.tokenize(con) for con in preprocessed_train['conversation'].tolist()]
tokenized_test = [tokenizer.tokenize(con) for con in preprocessed_test['text'].tolist()]

## [CLS], [SEP] 토큰 추가
token_id_train = add_special_tokens(tokenized_train)
token_id_test = add_special_tokens(tokenized_test)

# 패딩과 트렁케이션 설정
## 문장의 max length를 test 문장의 최대 길이로 설정한다. 
## padding은 앞에서부터 진행
## max길이를 넘어가면 앞에서부터 자른다. -> 한국말은 끝까지 들어야하므로..
token_id_train = pad_sequences(token_id_train, maxlen=191, dtype='long', padding='pre', truncating='pre', value=PAD_TOKEN_ID)
token_id_test = pad_sequences(token_id_test, maxlen=191, dtype='long', padding='pre', truncating='pre', value=PAD_TOKEN_ID)

In [73]:
tokenizer.vocab_size

119547